# PA1: Symbolic Execution on Deep Neural Networks

In [1]:
import z3

In [2]:
# Define the ReLU function 
def relu(v):
    return z3.If(v <= 0.0, 0.0, v)

# A 2X2X2X2 DNN

In [3]:
#Give symbolic inputs for a 2X2X2X2 Dnn
i0 = z3.Real('i0')
i1= z3.Real('i1')
n0_0 = z3.Real('n0_0')
n0_1 = z3.Real('n0_1')
n1_0 = z3.Real('n1_0')
n1_1 = z3.Real('n1_1')
o0= z3.Real('o0')
o1 = z3.Real('o1')

In [4]:
#Create dnn with 2 hidden layers 
hidden_layer0 = [(n0_0,[1.0, -1.0], 0, True),(n0_1,[1.0, 1.0], 0.0, True)]
hidden_layer1 = [(n1_0,[0.5, -0.2], 0.0, True) , (n1_1,[-0.5, 0.1], 0.0, True)]
output_layer =[(o0,[1.0, -1.0], 0.0, False),(o1,[-1.0, 1.0], 0.0, False)]
dnn = [hidden_layer0,hidden_layer1,output_layer]
print(dnn)
input_var = [i0,i1] 

[[(n0_0, [1.0, -1.0], 0, True), (n0_1, [1.0, 1.0], 0.0, True)], [(n1_0, [0.5, -0.2], 0.0, True), (n1_1, [-0.5, 0.1], 0.0, True)], [(o0, [1.0, -1.0], 0.0, False), (o1, [-1.0, 1.0], 0.0, False)]]


In [5]:
#Function helps in creating an array of neuron input values
def create_neurons(dnn,input_var):
    neurons=[]
    neurons.append(input_var)
    for i,layer in enumerate(dnn):
        temp_neurons=[]
        for j,(neuron, weight, b, r) in enumerate(layer):
            temp_neurons.append(dnn[i][j][0])
        neurons.append(temp_neurons)
    return(neurons)
        
        
create_neurons(dnn,input_var)
            
    

[[i0, i1], [n0_0, n0_1], [n1_0, n1_1], [o0, o1]]

# my_symbolic_execution Function

In [6]:
#Function calls the create_neuron function and uses it's return to calculate the weights of every possible neuron
def my_symbolic_execution(input_var, dnn):
#     weighted_sum = []
    neurons = create_neurons(dnn,input_var)
    res = []
    for i, layer in enumerate(dnn):
#         new_weights = []  
        for j, (neuron, weight, b, r) in enumerate(layer):
            w_sum = sum(var * w for var, w in zip(neurons[i], weight))            
            if b!= 0: #bias
                w_sum += b
                print(w_sum)
            if r:    #relu activation
                w_sum = relu(w_sum)
            res.append(neuron == w_sum)
    return(z3.And(res))



# Symbolic States and Constraint Solving

In [7]:
#Main
if __name__ == "__main__":
    states = my_symbolic_execution(input_var,dnn)
    print(states)
    assert z3.is_expr(states)
    z3.solve(states) #Generating random outputs

And(n0_0 == If(0 + i0*1 + i1*-1 <= 0, 0, 0 + i0*1 + i1*-1),
    n0_1 == If(0 + i0*1 + i1*1 <= 0, 0, 0 + i0*1 + i1*1),
    n1_0 ==
    If(0 + n0_0*1/2 + n0_1*-1/5 <= 0,
       0,
       0 + n0_0*1/2 + n0_1*-1/5),
    n1_1 ==
    If(0 + n0_0*-1/2 + n0_1*1/10 <= 0,
       0,
       0 + n0_0*-1/2 + n0_1*1/10),
    o0 == 0 + n1_0*1 + n1_1*-1,
    o1 == 0 + n1_0*-1 + n1_1*1)
[i1 = 4,
 n0_0 = 2,
 n1_1 = 0,
 i0 = 6,
 n0_1 = 10,
 o1 = 0,
 n1_0 = 0,
 o0 = 0]


In [8]:
# Stimutating concrete example
g = z3.And([i0 == 1.0, i1 == -1.0])
z3.solve(z3.And(states, g))  # you should get o0, o1 = 1, -1

[n1_1 = 0,
 i0 = 1,
 n0_1 = 0,
 o1 = -1,
 o0 = 1,
 n0_0 = 2,
 i1 = -1,
 n1_0 = 1]


# Checking Assertions

In [9]:
print("Prove that if (n0_0 > 0.0 and n0_1 <= 0.0) then o0 > o1")
g = z3.Implies(z3.And([n0_0 > 0.0, n0_1 <= 0.0]), o0 > o1)
print(g)  #  Implies (And(n0_0 > 0, n0_1 <= 0), o0 > o1)
z3.prove(z3.Implies(states, g))  # proved

Prove that if (n0_0 > 0.0 and n0_1 <= 0.0) then o0 > o1
Implies(And(n0_0 > 0, n0_1 <= 0), o0 > o1)
proved


In [10]:
print("Prove that when (i0 - i1 > 0 and i0 + i1 <= 0), then o0 > o1")
g = z3.Implies(z3.And([i0 - i1 > 0.0, i0 + i1 <= 0.0]), o0 > o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(states, g))
# proved

Prove that when (i0 - i1 > 0 and i0 + i1 <= 0), then o0 > o1
Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
proved


In [11]:
print("Disprove that when i0 - i1 >0, then o0 > o1")
g = z3.Implies(i0 - i1 > 0.0, o0 > o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(states, g))

Disprove that when i0 - i1 >0, then o0 > o1
Implies(i0 - i1 > 0, o0 > o1)
counterexample
[n1_1 = 0,
 i0 = 6,
 n0_1 = 10,
 o1 = 0,
 i1 = 4,
 n1_0 = 0,
 o0 = 0,
 n0_0 = 2]


# Own Example of a 2X2X2X2X2 DNN

In [12]:
#Give symbolic inputs for a 2X2X2X2X2 Dnn
i0 = z3.Real('i0')
i1= z3.Real('i1')
n0_0 = z3.Real('n0_0')
n0_1 = z3.Real('n0_1')
n1_0 = z3.Real('n1_0')
n1_1 = z3.Real('n1_1')
n2_0 = z3.Real('n2_0')
n2_1 = z3.Real('n2_1')
o0= z3.Real('o0')
o1 = z3.Real('o1')

In [13]:
#Create dnn with 3 hidden layers with 2 neurons
hidden_layer0 = [(n0_0,[1.0, -1.0], 0, True),(n0_1,[1.0, 1.0], 0.0, True)]
hidden_layer1 = [(n1_0,[0.5, -0.2], 0.0, True) , (n1_1,[-0.5, 0.1], 0.0, True)]
hidden_layer2 = [(n2_0,[1, 1], 0.0, True) , (n2_1,[1, 1], 0.0, True)]
output_layer =[(o0,[1.0, -1.0], 0.0, False),(o1,[-1.0, 1.0], 0.0, False)]
dnn = [hidden_layer0,hidden_layer1,hidden_layer2,output_layer]
print(dnn)
input_var = [i0,i1] 

[[(n0_0, [1.0, -1.0], 0, True), (n0_1, [1.0, 1.0], 0.0, True)], [(n1_0, [0.5, -0.2], 0.0, True), (n1_1, [-0.5, 0.1], 0.0, True)], [(n2_0, [1, 1], 0.0, True), (n2_1, [1, 1], 0.0, True)], [(o0, [1.0, -1.0], 0.0, False), (o1, [-1.0, 1.0], 0.0, False)]]


In [14]:
#Function helps in creating an array of neuron input values
def create_neurons(dnn,input_var):
    neurons=[]
    neurons.append(input_var)
    for i,layer in enumerate(dnn):
        temp_neurons=[]
        for j,(neuron, weight, b, r) in enumerate(layer):
            temp_neurons.append(dnn[i][j][0])
        neurons.append(temp_neurons)
    return(neurons)
        
        
create_neurons(dnn,input_var)

[[i0, i1], [n0_0, n0_1], [n1_0, n1_1], [n2_0, n2_1], [o0, o1]]

In [15]:
#Function calls the create_neuron function and uses it's return to calculate the weights of every possible neuron
def my_symbolic_execution(input_var, dnn):
    weighted_sum = []
    neurons = create_neurons(dnn,input_var)
    res = []
    for i, layer in enumerate(dnn):
        new_weights = []  
        for j, (neuron, weight, b, r) in enumerate(layer):
            w_sum = sum(var * w for var, w in zip(neurons[i], weight))            
            if b!= 0:
                w_sum += b
                print(w_sum)
            if r:
                w_sum = relu(w_sum)
            res.append(neuron == w_sum)
    return(z3.And(res))



In [16]:
#Main Function
if __name__ == "__main__":
    states = my_symbolic_execution(input_var,dnn)
    print(states)
    assert z3.is_expr(states)
    z3.solve(states) #Generating random outputs
    

And(n0_0 == If(0 + i0*1 + i1*-1 <= 0, 0, 0 + i0*1 + i1*-1),
    n0_1 == If(0 + i0*1 + i1*1 <= 0, 0, 0 + i0*1 + i1*1),
    n1_0 ==
    If(0 + n0_0*1/2 + n0_1*-1/5 <= 0,
       0,
       0 + n0_0*1/2 + n0_1*-1/5),
    n1_1 ==
    If(0 + n0_0*-1/2 + n0_1*1/10 <= 0,
       0,
       0 + n0_0*-1/2 + n0_1*1/10),
    n2_0 ==
    If(0 + n1_0*1 + n1_1*1 <= 0, 0, 0 + n1_0*1 + n1_1*1),
    n2_1 ==
    If(0 + n1_0*1 + n1_1*1 <= 0, 0, 0 + n1_0*1 + n1_1*1),
    o0 == 0 + n2_0*1 + n2_1*-1,
    o1 == 0 + n2_0*-1 + n2_1*1)
[n1_0 = 0,
 o0 = 0,
 i0 = 4,
 n0_1 = 15/2,
 n2_0 = 1/2,
 o1 = 0,
 n0_0 = 1/2,
 i1 = 7/2,
 n2_1 = 1/2,
 n1_1 = 1/2]


In [17]:
# Stimutating concrete example
g = z3.And([i0 == 1.0, i1 == -1.0])
z3.solve(z3.And(states, g))  # you should get o0, o1 = 1, -1

[n2_1 = 1,
 i1 = -1,
 n1_1 = 0,
 n1_0 = 1,
 i0 = 1,
 n0_0 = 2,
 n2_0 = 1,
 n0_1 = 0,
 o1 = 0,
 o0 = 0]


In [18]:
print("Prove that if (n0_0 > 0.0 and n0_1 <= 0.0) then o0 > o1")
g = z3.Implies(z3.And([n0_0 > 0.0, n0_1 <= 0.0]), o0 > o1)
print(g)  #  Implies (And(n0_0 > 0, n0_1 <= 0), o0 > o1)
z3.prove(z3.Implies(states, g))  # proved

Prove that if (n0_0 > 0.0 and n0_1 <= 0.0) then o0 > o1
Implies(And(n0_0 > 0, n0_1 <= 0), o0 > o1)
counterexample
[n2_0 = 1/4,
 i0 = 1/8,
 n0_0 = 1/2,
 i1 = -3/8,
 n2_1 = 1/4,
 n1_0 = 1/4,
 o1 = 0,
 n1_1 = 0,
 n0_1 = 0,
 o0 = 0]


In [19]:
print("Prove that when (i0 - i1 > 0 and i0 + i1 <= 0), then o0 > o1")
g = z3.Implies(z3.And([i0 - i1 > 0.0, i0 + i1 <= 0.0]), o0 > o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(states, g))

Prove that when (i0 - i1 > 0 and i0 + i1 <= 0), then o0 > o1
Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
counterexample
[n2_1 = 1/4,
 i1 = -1/2,
 n1_1 = 0,
 n1_0 = 1/4,
 i0 = 0,
 o0 = 0,
 n2_0 = 1/4,
 n0_1 = 0,
 o1 = 0,
 n0_0 = 1/2]


# A 2X4X4X4X2 DNN

In [20]:
#Give symbolic inputs for a 2X4X4X4X2 Dnn
i0 = z3.Real('i0')
i1= z3.Real('i1')
n0_0 = z3.Real('n0_0')
n0_1 = z3.Real('n0_1')
n0_2 = z3.Real('n0_2')
n0_3 = z3.Real('n0_3')
n1_0 = z3.Real('n1_0')
n1_1 = z3.Real('n1_1')
n1_2 = z3.Real('n1_2')
n1_3 = z3.Real('n1_3')
n2_0 = z3.Real('n2_0')
n2_1 = z3.Real('n2_1')
n2_2 = z3.Real('n2_2')
n2_3 = z3.Real('n2_3')
o0= z3.Real('o0')
o1 = z3.Real('o1')

In [21]:
#Create dnn with 3 hidden layers each having 4 neurons with non-zero bias values
hidden_layer0 = [(n0_0,[1.0, -1.0], 1, True),(n0_1,[1.0, 1.0], 1, True),(n0_2,[1, 1], 1, True),(n0_3,[1, 1], 1, True)]
hidden_layer1 = [(n1_0,[0.5, -0.2], 1, True) , (n1_1,[-0.5, 0.1], 1, True),(n1_2,[1, 1], 0.0, True),(n1_3,[1, 1], 1, True)]
hidden_layer2 = [(n2_0,[1, 1], 1, True) , (n2_1,[1, 1], 1, True),(n2_2,[1, 1], 1, True),(n2_3,[1, 1], 1, True)]
output_layer =[(o0,[1.0, -1.0], 1.0, False),(o1,[-1.0, 1.0], 1.0, False)]
dnn = [hidden_layer0,hidden_layer1,hidden_layer2,output_layer]
print(dnn)
input_var = [i0,i1] 

[[(n0_0, [1.0, -1.0], 1, True), (n0_1, [1.0, 1.0], 1, True), (n0_2, [1, 1], 1, True), (n0_3, [1, 1], 1, True)], [(n1_0, [0.5, -0.2], 1, True), (n1_1, [-0.5, 0.1], 1, True), (n1_2, [1, 1], 0.0, True), (n1_3, [1, 1], 1, True)], [(n2_0, [1, 1], 1, True), (n2_1, [1, 1], 1, True), (n2_2, [1, 1], 1, True), (n2_3, [1, 1], 1, True)], [(o0, [1.0, -1.0], 1.0, False), (o1, [-1.0, 1.0], 1.0, False)]]


In [22]:
#Function helps in creating an array of neuron input values
def create_neurons(dnn,input_var):
    neurons=[]
    neurons.append(input_var)
    for i,layer in enumerate(dnn):
        temp_neurons=[]
        for j,(neuron, weight, b, r) in enumerate(layer):
            temp_neurons.append(dnn[i][j][0])
        neurons.append(temp_neurons)
    return(neurons)
        
        
create_neurons(dnn,input_var)

[[i0, i1],
 [n0_0, n0_1, n0_2, n0_3],
 [n1_0, n1_1, n1_2, n1_3],
 [n2_0, n2_1, n2_2, n2_3],
 [o0, o1]]

In [23]:
#Function calls the create_neuron function and uses it's return to calculate the weights of every possible neuron
def my_symbolic_execution(input_var, dnn):
    weighted_sum = []
    neurons = create_neurons(dnn,input_var)
    res = []
    for i, layer in enumerate(dnn):
        new_weights = []  
        for j, (neuron, weight, b, r) in enumerate(layer):
            w_sum = sum(var * w for var, w in zip(neurons[i], weight))            
            if b!= 0:
                w_sum += b
                print(w_sum)
            if r:
                w_sum = relu(w_sum)
            res.append(neuron == w_sum)
    return(z3.And(res))



In [24]:
#Main
if __name__ == "__main__":
    states = my_symbolic_execution(input_var,dnn)
    print(states)
    assert z3.is_expr(states)
    z3.solve(states) #Generating random outputs
    

0 + i0*1 + i1*-1 + 1
0 + i0*1 + i1*1 + 1
0 + i0*1 + i1*1 + 1
0 + i0*1 + i1*1 + 1
0 + n0_0*1/2 + n0_1*-1/5 + 1
0 + n0_0*-1/2 + n0_1*1/10 + 1
0 + n0_0*1 + n0_1*1 + 1
0 + n1_0*1 + n1_1*1 + 1
0 + n1_0*1 + n1_1*1 + 1
0 + n1_0*1 + n1_1*1 + 1
0 + n1_0*1 + n1_1*1 + 1
0 + n2_0*1 + n2_1*-1 + 1
0 + n2_0*-1 + n2_1*1 + 1
And(n0_0 ==
    If(0 + i0*1 + i1*-1 + 1 <= 0, 0, 0 + i0*1 + i1*-1 + 1),
    n0_1 ==
    If(0 + i0*1 + i1*1 + 1 <= 0, 0, 0 + i0*1 + i1*1 + 1),
    n0_2 ==
    If(0 + i0*1 + i1*1 + 1 <= 0, 0, 0 + i0*1 + i1*1 + 1),
    n0_3 ==
    If(0 + i0*1 + i1*1 + 1 <= 0, 0, 0 + i0*1 + i1*1 + 1),
    n1_0 ==
    If(0 + n0_0*1/2 + n0_1*-1/5 + 1 <= 0,
       0,
       0 + n0_0*1/2 + n0_1*-1/5 + 1),
    n1_1 ==
    If(0 + n0_0*-1/2 + n0_1*1/10 + 1 <= 0,
       0,
       0 + n0_0*-1/2 + n0_1*1/10 + 1),
    n1_2 ==
    If(0 + n0_0*1 + n0_1*1 <= 0, 0, 0 + n0_0*1 + n0_1*1),
    n1_3 ==
    If(0 + n0_0*1 + n0_1*1 + 1 <= 0,
       0,
       0 + n0_0*1 + n0_1*1 + 1),
    n2_0 ==
    If(0 + n1_0*1 + n1_1*1 +

In [25]:
#  Stimutating concrete example
g = z3.And([i0 == 1.0, i1 == -1.0])
z3.solve(z3.And(states, g))  # you should get o0, o1 = 1, -1
    

[n1_0 = 23/10,
 i0 = 1,
 n2_2 = 33/10,
 o1 = 1,
 n1_2 = 4,
 n2_3 = 33/10,
 o0 = 1,
 n0_0 = 3,
 i1 = -1,
 n0_2 = 1,
 n2_1 = 33/10,
 n0_3 = 1,
 n1_1 = 0,
 n2_0 = 33/10,
 n0_1 = 1,
 n1_3 = 5]


In [26]:
#  Stimutating concrete example
g = z3.And([i0 == 1.0, i1 == -0.5])
z3.solve(z3.And(states, g))  # you should get o0, o1 = 1, -1
    

[n1_0 = 39/20,
 o1 = 1,
 i0 = 1,
 n2_2 = 59/20,
 n1_2 = 4,
 o0 = 1,
 n2_3 = 59/20,
 n0_0 = 5/2,
 i1 = -1/2,
 n0_2 = 3/2,
 n1_3 = 5,
 n0_3 = 3/2,
 n1_1 = 0,
 n2_0 = 59/20,
 n2_1 = 59/20,
 n0_1 = 3/2]


# Checking Assertions - 2 True & 1 False

In [27]:
print("Prove that if (n0_0 > 0.0 and n0_1 < 0.0) then o0 > o1")
g = z3.Implies(z3.And([n0_0 > 0.0, n0_1 < 0.0]), o0 > o1)
print(g)  #  Implies (And(n0_0 > 0, n0_1 <= 0), o0 > o1)
z3.prove(z3.Implies(states, g))  # proved

Prove that if (n0_0 > 0.0 and n0_1 < 0.0) then o0 > o1
Implies(And(n0_0 > 0, n0_1 < 0), o0 > o1)
proved


In [28]:
print("Prove that when (i0 - i1 > 0 and i0 + i1 <= 0), then o0 > o1")
g = z3.Implies(z3.And([i0 - i1 > 0.0, i0 + i1 <= 0.0]), o0 > o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(states, g))

Prove that when (i0 - i1 > 0 and i0 + i1 <= 0), then o0 > o1
Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
counterexample
[o1 = 1,
 i0 = -1/14,
 n2_2 = 167/56,
 n1_2 = 13/7,
 o0 = 1,
 n2_3 = 167/56,
 n0_0 = 47/28,
 n1_0 = 101/56,
 n0_2 = 5/28,
 i1 = -3/4,
 n0_3 = 5/28,
 n2_1 = 167/56,
 n1_1 = 5/28,
 n2_0 = 167/56,
 n0_1 = 5/28,
 n1_3 = 20/7]


In [29]:
print("Prove that when i0 == 1.0, i1 == -1.0, then o0 is equal to o1")
g = z3.Implies(z3.And([i0 == 1.0, i1 == -1.0]), o0 == o1)
print(g)  # Implies(And(i0 - i1 > 0, i0 + i1 <= 0), o0 > o1)
z3.prove(z3.Implies(states, g))

Prove that when i0 == 1.0, i1 == -1.0, then o0 is equal to o1
Implies(And(i0 == 1, i1 == -1), o0 == o1)
proved
